In [16]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import os
import kagglehub
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


# Download latest version
path = kagglehub.dataset_download("arshid/iris-flower-dataset")
# df = pd.read_csv(path + "Iris.csv")
df = pd.read_csv(os.path.join(path, "Iris.csv"))
# print("Path to dataset files:", path)

In [17]:
X = df[["sepal_length", "sepal_width", "petal_length", "petal_width"]].values
y = df["species"].values
# print(X.shape)
# y = y.reshape(150, 1)
# print(y)
encoder = LabelEncoder()
y_encoder = encoder.fit_transform(y)
y_encoder = y_encoder.reshape(-1, 1)
oneHot = OneHotEncoder(sparse_output = False)
y_encoder = oneHot.fit_transform(y_encoder) #shape (150,3)
np.random.seed(42)
w1 = np.random.randn(4, 64) * 0.01
b1 = np.zeros((1, 64))
w2 = np.random.randn(64, 3) * 0.01
b2 = np.zeros((1, 3))
# b1 = 0
# b2 = 0
# b3 = 0

a = 0.1
# print(y_encoder)

In [18]:
# def sigmoid(z):
#     return (1/(1+np.exp(-z)))
def ReLU(z):
    return np.maximum(0, z)
def ReLU_Derivative(z):
    return (z>0).astype(float)
def sigmoid(z):
    return (1/(1+np.exp(-z)))
def softmax(z): #vectorized
        exp_values = np.exp(z-np.max(z, axis = 1, keepdims = True)) #tim gia tri lon nhat cua moi sample  roi tru di so lon nhat
        return exp_values/np.sum(exp_values, axis = 1, keepdims = True) #tong cong trong hang, tranh overflow

# def Dense(X, w, b, activation=None):
#     Z = np.dot(X, w) + b
#     if activation == "relu":
#         return ReLU(Z)
#     elif activation == "sigmoid":
#         return sigmoid(Z)
#     elif activation == "softmax":
#         return softmax(Z)
#     else:
#         return Z  # linear output


In [22]:
def cross_entropy(y_true, y_pred):
    m = y_true.shape[0]
    return (-1/m) * np.sum(y_true * np.log(y_pred+1e-9))
for epoch in range(1000):
    z1 = np.dot(X, w1) + b1
    A1 = ReLU(z1)
# for _ in range(64):
    z2 = np.dot(A1, w2) + b2
    A2 = softmax(z2)
    # error = A2 - y
    # dw2 = np.dot()
    # Now, let's implement back propagation
    loss = cross_entropy(y_encoder, A2)
    m = X.shape[0]
    dz2 = (A2 - y_encoder)/m #Let's have the prediction substract the real label, so we could have the error
    dw2 = np.dot(A1.T, dz2) #Once done, we will have an dot product with the dz2 from A1.Trans to know how much hidden layers has been impacted
    db2 = np.sum(dz2, axis = 0, keepdims = True) #How much errors that have been summed up in a column sum
    dA1 =  np.dot(dz2, w2.T) #Once done, we will have an dot product with w2.T and dz2 to see how much A1 has been affected
    dz1 = dA1 * ReLU_Derivative(z1) # If we're hidden when we're using ReLU, it only activates when neuron takes a fault
    dw1 = np.dot(X.T, dz1) #Same as the above.
    db1 = np.sum(dz1, axis=0, keepdims=True)#Same as the above
# The procedure is: If the output is wrong -> Split those errors back into layer and neuron -> And, continue until we gain gradient on every number.
    w1 -= a*dw1
    b1 -= a*db1
    w2 -= a*dw2
    b2-= a*db2
    if epoch % 100 == 0:
        preds = np.argmax(A2, axis=1)
        labels = np.argmax(y_encoder, axis=1)   # ✅ chuyển từ one-hot
        acc = np.mean(preds == labels)
        print(f"Epoch {epoch} | Loss: {loss:.4f} | Accuracy: {acc:.4f}")

    
preds = np.argmax(A2, axis=1)
labels = np.argmax(y_encoder, axis=1)
accuracy = np.mean(preds == labels)
print("Final accuracy:", accuracy)


Epoch 0 | Loss: 1.0984 | Accuracy: 0.3333
Epoch 100 | Loss: 0.3117 | Accuracy: 0.9733
Epoch 200 | Loss: 0.2858 | Accuracy: 0.8667
Epoch 300 | Loss: 0.1899 | Accuracy: 0.9200
Epoch 400 | Loss: 0.1286 | Accuracy: 0.9333
Epoch 500 | Loss: 0.1107 | Accuracy: 0.9333
Epoch 600 | Loss: 0.0943 | Accuracy: 0.9467
Epoch 700 | Loss: 0.0808 | Accuracy: 0.9467
Epoch 800 | Loss: 0.0783 | Accuracy: 0.9600
Epoch 900 | Loss: 0.0749 | Accuracy: 0.9600
Final accuracy: 0.9733333333333334
